In [1]:
import pysam
from pysam import VariantFile as vcf
import operator
from math import log2
import pandas as pd
from pandas import DataFrame as dataframe
import matplotlib.pyplot as plt
import numpy as np
from scipy.spatial.distance import pdist, squareform
import scipy
import  os
import os.path
import matplotlib.colors as mcolors
from scipy import stats
import csv
import requests
import re
import json


In [2]:

split_race_region={'ACB': {},
 'ASW': {},
 'BEB': {'split_chr3/xav': [(98046342, 98273693)]},
 'CDX': {'split_chr13/xai': [(55102733, 55354096)]},
 'CEU': {'split_chr17/xaj': [(45670177, 46418024)]},
 'CHB': {},
 'CHS': {'split_chr3/xav': [(98046342, 98245327)]},
 'CLM': {'split_chr3/xav': [(98046342, 98273693)]},
 'ESN': {'split_chr5/xai': [(34311870, 34562837)]},
 'FIN': {},
 'GBR': {'split_chr17/xaj': [(45670177, 46418024)]},
 'GIH': {'split_chr3/xav': [(98046342, 98276757)]},
 'GWD': {},
 'IBS': {'split_chr17/xaj': [(45836662, 46418024)]},
 'ITU': {'split_chr17/xak': [(52575391, 52889013)]},
 'JPT': {},
 'KHV': {},
 'LWK': {},
 'MSL': {},
 'MXL': {'split_chr3/xav': [(98046342, 98217489)]},
 'PEL': {'split_chr11/xaf': [(23168664, 23461651)],
  'split_chr3/xav': [(98046342, 98217489)]},
 'PJL': {},
 'PUR': {'split_chr3/xav': [(98046342, 98273693)]},
 'STU': {'split_chr13/xai': [(55102733, 55443491)],
  'split_chr3/xav': [(98046342, 98273693)]},
 'TSI': {'split_chr17/xaj': [(45836662, 46418024)]},
 'YRI': {}}

In [3]:
def fetch_gene_info(chromosome, start, end):
    query_xml = f"""
    <Query virtualSchemaName="default" formatter="TSV" header="0" uniqueRows="1" count="" datasetConfigVersion="0.6">
        <Dataset name="hsapiens_gene_ensembl" interface="default">
            <Filter name="chromosome_name" value="{chromosome}"/>
            <Filter name="start" value="{start}"/>
            <Filter name="end" value="{end}"/>
            <Attribute name="ensembl_gene_id"/>
            <Attribute name="external_gene_name"/>
            <Attribute name="description"/>
            <Attribute name="start_position"/>
            <Attribute name="end_position"/>            
            <Attribute name="strand"/>
        </Dataset>
    </Query>
    """

    biomart_url = "http://www.ensembl.org/biomart/martservice?query="
    response = requests.get(biomart_url + query_xml.strip())

    if response.status_code != 200:
        raise Exception(f"Error fetching data from BioMart: {response.text}")

    genes = [line.split("\t") for line in response.text.strip().split("\n")]
    return genes

In [4]:
def generate_gene_info(gene_list_one_record,chrnum):
    result=[]
    result.append(gene_list_one_record[0])
    result.append(gene_list_one_record[1])
    s=gene_list_one_record[3]
    e=gene_list_one_record[4]
    
    pos="chr"+chrnum+":"+str(s)+"-"+str(e)
    result.append(pos)
    result.append(gene_list_one_record[5])
    result.append(gene_list_one_record[2])
    
    return result

In [5]:

def extract_numbers(input_string):
    return re.findall(r'\d+', input_string)

def ifgeneinside_region(complementary_region,gene_region):
    
    s=int(gene_region[0])
    e=int(gene_region[1])
     
    if (complementary_region[0]<=s) and complementary_region[1]>=e:
        return True
    else:
        return False

In [6]:
allrecords=[]

#non_mhc
for race,split_ in split_race_region.items():
    if split_ !={}:
        if race!="All":
            for chr,arr in split_.items():
                chr=chr.split("/")[0].split("_")[1]
                chr=extract_numbers(chr)[0]
                print(chr)
                for a in arr:
                    gene_list=fetch_gene_info(chromosome=chr,start=a[0]-1,end=a[1]-1)
                    for gene_list_record in gene_list:
                        if ifgeneinside_region(complementary_region=a,gene_region=[gene_list_record[3],gene_list_record[4]]):
                            onerecord=[race,0]
                            onerecord+=["chr"+chr+":"+str(a[0])+"-"+str(a[1])]
                            
                            onerecord+=generate_gene_info(gene_list_one_record=gene_list_record,chrnum=chr)
                            allrecords.append(onerecord)
                        else:
                            continue

3
13
17
3
3
5
17
3
17
17
3
11
3
3
13
3
17


In [7]:
def generatedf(columns,allrecords):
    dictforDF=dict()
    for i in range(len(columns)):
        midarr=[]
        for record in allrecords:           
            midarr.append(record[i])
        dictforDF[columns[i]]=midarr
    #dataframe(dictforDF).to_csv("csv2_05_09.csv")
    return dataframe(dictforDF)

dfnew=generatedf(columns=["Race","isMHC","Position of disassortative mating region","Gene ID","Gene name","Position of gene","is_complement","Gene description"],allrecords=allrecords)



In [8]:
dfnew

,Race,isMHC,Position of disassortative mating region,Gene ID,Gene name,Position of gene,is_complement,Gene description
0,BEB,0,chr3:98046342-98273693,ENSG00000250750,OR5BM1P,chr3:98053374-98053739,1,olfactory receptor family 5 subfamily BM membe...
1,BEB,0,chr3:98046342-98273693,ENSG00000213439,OR5AC1,chr3:98064472-98065396,1,olfactory receptor family 5 subfamily AC membe...
2,BEB,0,chr3:98046342-98273693,ENSG00000196578,OR5AC2,chr3:98087173-98088102,1,olfactory receptor family 5 subfamily AC membe...
3,BEB,0,chr3:98046342-98273693,ENSG00000249225,,chr3:98099908-98148134,-1,"novel transcript, antisense to OR5H1, OR5H14, ..."
4,BEB,0,chr3:98046342-98273693,ENSG00000290819,,chr3:98100793-98105672,1,novel transcript
...,...,...,...,...,...,...,...,...
205,TSI,0,chr17:45836662-46418024,ENSG00000261575,,chr17:46267412-46268694,1,C17orf58 pseudogene
206,TSI,0,chr17:45836662-46418024,ENSG00000228696,ARL17B,chr17:46274784-46361766,-1,ADP ribosylation factor like GTPase 17B [Sourc...
207,TSI,0,chr17:45836662-46418024,ENSG00000289729,,chr17:46283310-46286966,-1,bromodomain PHD finger transcription factor (B...
208,TSI,0,chr17:45836662-46418024,ENSG00000176681,LRRC37A,chr17:46292733-46338077,1,leucine rich repeat containing 37A [Source:HGN...


In [9]:
dfnew.to_csv("split.csv")